In [ ]:
# LABORATORIUM 4 - ZADANIE 1
# IMIĘ i NAZWISKO: MICHAŁ WÓJCIK

In [ ]:
# Zaimportowanie niezbędnych bibliotek
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, RocCurveDisplay, roc_curve

# Ustawienie rozmiarów obrazów
img_width, img_height = 224, 224

In [ ]:
# Ustawienie ścieżek do folderów z danymi oraz ilosci próbek, epok i rozmiaru batcha
train_data_dir_path = 'train'
validation_data_dir_path = 'test'
nb_train_samples =400
nb_validation_samples = 100
epochs = 10
batch_size = 16

In [ ]:
# Ustawienie wymiarów wejściowych w zależności od formatu obrazu
if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else: # channels_last
    input_shape = (img_width, img_height, 3)

In [ ]:
# Stworzenie modelu konwolucyjnej sieci neuronowej (CNN)
# Składa się z trzech bloków konwolucyjych, a każdy blok składa się z warstwy konwolucyjnej, warstwy aktywacji i warstwy pooling.
model = Sequential()
model.add(Conv2D(32, (2, 2), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (2, 2)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

In [ ]:
# Zbudowanie (kompilacja) modelu
model.compile(loss='binary_crossentropy',
			optimizer='rmsprop',
			metrics=['accuracy'])

In [ ]:
# Augmentacja danych wejściowych - odbywa się dopiero w trakcie trenowania modelu
train_data_generator = ImageDataGenerator(
	rescale=1. / 255,
	shear_range=0.2,
	zoom_range=0.2,
	horizontal_flip=True)

test_data_generator = ImageDataGenerator(rescale=1. / 255)

train_generator = train_data_generator.flow_from_directory(
	train_data_dir_path,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

validation_generator = test_data_generator.flow_from_directory(
	validation_data_dir_path,
	target_size=(img_width, img_height),
	batch_size=batch_size,
	class_mode='binary')

# Trenowanie modelu
model_history = model.fit(
	train_generator,
	steps_per_epoch=nb_train_samples // batch_size,
	epochs=epochs,
	validation_data=validation_generator,
	validation_steps=nb_validation_samples // batch_size)

In [ ]:
# Wykres wartości funkcji straty dla zbioru treningowego i walidacyjnego
plt.figure(constrained_layout=True)
plt.plot(model_history.history['loss'], 'g')
plt.plot(model_history.history['val_loss'], 'r')
plt.title('model Loss')
plt.ylabel('loss, val_loss')
plt.xlabel('epoch')
plt.legend(['loss', 'val_loss'])

In [ ]:
# Stworzenie tablic przechowujących wartości predykcji, wartości rzeczywistych i wyniku przewidywania
predClass = []
actClass = []
predClassScore = []

# Przewidywanie klas dla obrazów z folderu testowego
for i in range(41, 51):
	actClass.append(0)
	print('test/cars/' + str(i) + '.jpg')
	image = load_img('test/cars/' + str(i) + '.jpg', target_size=(224, 224))
	img = np.array(image)
	img = img / 255.0
	img = img.reshape(1, 224, 224, 3)
	label = model.predict(img)
	
	print("Predicted Class (0 - Cars, 1 - Planes): ", label[0][0])
	
	if label[0][0] < 0.5:
		predClass.append(0)
	else:
		predClass.append(1)
		
	predClassScore.append(label[0][0])
	
for i in range(41, 51):
	actClass.append(1)
	print('test/planes/' + str(i) + '.jpg')
	image = load_img('test/planes/' + str(i) + '.jpg', target_size=(224, 224))
	img = np.array(image)
	img = img / 255.0
	img = img.reshape(1, 224, 224, 3)
	label = model.predict(img)
	
	print("Predicted Class (0 - Cars, 1 - Planes): ", label[0][0])
	
	if label[0][0] < 0.5:
		predClass.append(0)
	else:
		predClass.append(1)
		
	predClassScore.append(label[0][0])

In [ ]:
# LABORATORIUM 4 - ZADANIE 2
# IMIĘ i NAZWISKO: MICHAŁ WÓJCIK

In [ ]:
# Wyznaczanie macierzy konfuzji
tn, fp, fn, tp = confusion_matrix(actClass, predClass).ravel()
# tn - true negative, fp - false positive, fn - false negative, tp - true positive
(tn, fp, fn, tp)

In [ ]:
# Wyświetlanie macierzy konfuzji
ConfusionMatrixDisplay.from_predictions(actClass, predClass)

In [ ]:
# Wyznaczanie i wyświetlanie krzywej ROC (sposob 1)
RocCurveDisplay.from_predictions(actClass, predClassScore)

In [ ]:
# Wyznaczanie i wyświetlanie krzywej ROC (sposób 2)
fpr, tpr, thresholds = roc_curve(actClass, predClassScore)
optimal_idx = np.argmin(np.abs(tpr[1:] - fpr[1:])) + 1
optimal_threshold = thresholds[optimal_idx]
print("Optymalny próg odcięcia: ", optimal_threshold)